In [14]:
import os
import django
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "activity_viewer.settings")
django.setup()

In [15]:
from dashboard.models import Log
from django.db.models import Count

In [16]:
bad_request_logs = Log.objects.filter(error_code__isnull=False)
most_common_errors = bad_request_logs.values_list('error_code').annotate(count=Count('error_code')).order_by('-count')
most_common_errors

<QuerySet [('InvalidRange', 208), ('NoSuchKey', 3), ('ExpiredToken', 1), ('InternalError', 1)]>

In [17]:
china_ip_errors = bad_request_logs.filter(ip_address='202.118.228.188').values_list('error_code').annotate(count=Count('error_code')).order_by('-count')
china_ip_errors

<QuerySet []>